# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [17]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [18]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [19]:
# Import the necessary CSVs to Pandas DataFrames
# Read the census data into a Pandas DataFrame
file_path = Path("sfo_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - -

In [20]:
# Load neighborhoods coordinates data
# Read the Mapbox API key
load_dotenv('sample.env')
map_box_api = os.getenv("mapbox")

# Set token using Plotly Express set function
px.set_mapbox_access_token(map_box_api)

# Read in data
sfo_map = pd.read_csv(Path("sfo_neighborhoods_coordinates.csv")).drop_duplicates()

sfo_map.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [22]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units_per_year = sfo_data["housing_units"]
    mean_housing_units_per_year = total_housing_units_per_year.groupby("year").mean()
        
    housing_units_fig = plt.figure()
    housing_units_plot = mean_housing_units_per_year.plot.bar(label=" Housing Units per Year", ylim=[370000, 385000])
    
    plt.close(housing_units_fig)
    
    return pn.pane.Matplotlib(housing_unit_fig)
    

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    gross_rent = sfo_data["gross_rent"]
    average_gross_rent = average_gross_rent.groupby("year").mean()

    gross_rent_fig = plt.figure()
    gross_rent_plot = avg_gross_rent.plot.line(label = "Average Gross Rent", line_color = "red")
    
    plt.close(gross_rent_fig)
    
    return pn.pane.Matplotlib(gross_rent_fig)


def average_sales_price():
    """Average Sales Price Per Year."""
    sale_price_sqt = sfo_data["sale_price_sqr_foot"]
    average_sales_price_sqt = sale_price_sqt.groupby("year").mean()
    
    avg_sales_fig = plt.figure()
    avg_sales_plot = average_sales_price_sqt.plot.line(label = "Average Sales Price per SQ Foot")
    
    plt.close(avg_sales_fig)
    
    return pn.pane.Matplotlib(avg_sales_fig)


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    average_prices_by_neighborhood = sfo_data[["neighborhood", "gross_rent"]]
    average_prices_by_neighborhood = average_prices_by_neighborhood.groupby("neighborhood").mean()
    

    average_prices_by_neighborhood_plot = average_prices_by_neighborhood.hvplot.line(label = "Average Prices by Neighborhood",rot=45, width =1000)
    
    return average_prices_by_neighborhood_plot


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_ten_by_neighborhood = sfo_data[["neighborhood", "gross_rent"]]
    top_ten_by_neighborhood = top_ten_by_neighborhood.groupby("neighborhood").mean().sort_values(by='gross_rent', ascending=False).iloc[:10]
    
    top_ten_by_neighborhood_plot = top_ten_by_neighborhood.hvplot.bar(label = "Top Ten Most Expensive Neighborhood", rot =45, width = 600)
    
    return top_ten_by_neighborhood_plot


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    expensive_neighborhoods = expensive_neighborhoods.sort_values(by="sale_price_sqr_foot", ascending=False)
    expensive_neighborhoods = expensive_neighborhoods.reset_index()

    expensive_neighborhood_plot = expensive_neighborhoods.hvplot(label= "Most Expensive Rent & Sales Comparison", rot=45)
    
    return expensive_neighborhood_plot

   
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    expensive_neighborhoods = expensive_neighborhoods.sort_values(by="sale_price_sqr_foot", ascending=False)
    expensive_neighborhoods = expensive_neighborhoods.reset_index()

    parallel_coordinates_plot = px.parallel_coordinates(expensive_neighborhoods, color='sale_price_sqr_foot')


    return parallel_coordinates_plot

  
def parallel_categories():
    """Parallel Categories Plot."""
    expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    expensive_neighborhoods = expensive_neighborhoods.sort_values(by="sale_price_sqr_foot", ascending=False)
    expensive_neighborhoods = expensive_neighborhoods.reset_index()
        
    parallel_categories_plot = px.parallel_categories(
    expensive_neighborhoods,
    dimensions = ['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
    title='Most Expensive Neighborhoods in San Fransisco',
    color='sale_price_sqr_foot',
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neighborhood": "Neighborhood",
        "sale_price_sqr_foot": "Sales Price Per SqFt",
        "housing_units": "Number of Houing Units",
        "gross_rent": "Gross Rent"})

    return parallel_categories_plot

    
def neighborhood_map():
    """Neighborhood Map."""
    average_prices_by_neighborhood = sfo_data[["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"]]
    average_prices_by_neighborhood = average_prices_by_neighborhood.groupby("neighborhood").mean()

    sfo_coordinates = pd.read_csv("sfo_neighborhoods_coordinates.csv").drop_duplicates()

    sfo_sales_map = (
    pd.concat([area_with_index, coordinates_with_index], axis=1, sort=True)
    .dropna()
    .reset_index()


    mapplot_neighborhood = px.scatter_mapbox(
    sfo_sales_map,
    lat="Lat",
    lon="Lon",
    color="gross_rent",
    text=None,
    hover_name="index",
    hover_data=None,
    custom_data=None,
    size="sale_price_sqr_foot",
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels="gross_rent",
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    opacity=None,
    size_max=None,
    zoom=12,
    center=None,
    mapbox_style="carto-positron",
    title="San Fransisco Sale Prices",
    template=None,
    width=1000,
    height=1000,
    
)
    plotly_panel = pn.pane.Plotly(mapplot_neighborhood)
    plotly_panel.updates = True
    return plotly_panel


def sunburst():
    """Sunburst Plot."""
    expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    expensive_neighborhoods = expensive_neighborhoods.sort_values(by="sale_price_sqr_foot", ascending=False)
    expensive_neighborhoods = expensive_neighborhoods.reset_index()
        
    expensive_neighborhoods_year = sfo_data[sfo_data["neighborhood"].isin(expensive_neighborhoods["neighborhood"])]
    expensive_neighborhoods_year = expensive_neighborhoods_year.reset_index()
    expensive_neighborhoods_year = expensive_neighborhoods_year

    sunburst_plot = px.sunburst(
    expensive_neighborhoods_year,
    path=['year', 'neighborhood'],
    title='Cost Analisys of Most Expensive Neigborhoods in San Fransisco',
    values='sale_price_sqr_foot',
    color='gross_rent',
    hover_data=['neighborhood'],
    height=700,
    width=700,
    color_continuous_scale='blues',
    color_continuous_midpoint=np.average(expensive_neighborhoods_year['gross_rent'],
    weights=expensive_neighborhoods_year['sale_price_sqr_foot']))

    return sunburst_plot


SyntaxError: invalid syntax (<ipython-input-22-2586197e0bdd>, line 120)

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!


# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [35]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units = sfo_data["housing_units"]
    housing_units = housing_units.groupby("year").mean()
        
    housing_units_fig = plt.figure()
    housing_units_plot = housing_units.plot.bar(label=" Housing Units per Year", ylim=[370000, 385000])
    
    plt.close(housing_units_fig)
    
    return pn.pane.Matplotlib(housing_unit_fig)
    

In [36]:
housing_units_per_year()

NameError: name 'housing_unit_fig' is not defined

In [37]:
average_gross_rent()

NameError: name 'average_gross_rent' is not defined

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()